In [1]:
import pandas as pd
import numpy as np
from wmb import *
from ALLCools.plot import *

import pathlib

CEMBA_SNM3C_3C_CHROM100K_RAW_ZARR_PATH do not exist: /gale/netapp/cemba3c/BICCN/CEMBA_3C/mcds/CEMBA.snm3C.chrom100k_raw.zarr
CEMBA_SNM3C_3C_COMPARTMENT_ZARR_PATH do not exist: /gale/netapp/cemba3c/BICCN/CEMBA_3C/mcds/CEMBA.snm3C.3C.mcds/chrom100k
CEMBA_SNM3C_3C_DOMAIN_INSULATION_ZARR_PATH do not exist: /gale/netapp/cemba3c/BICCN/CEMBA_3C/mcds/CEMBA.snm3C.3C.mcds/chrom25k


In [2]:
mc_annot = cemba.get_mc_annot()
rna_annot = broad.get_tenx_annot()

In [3]:
def save_index(cells, file_name, downsample=100000, random_state=0):
    cells = pd.Series(cells)
    if cells.size > downsample:
        cells = cells.sample(downsample, random_state=random_state)
    cells.to_csv(file_name, index=None, header=False)

In [4]:
rna_annot

<xarray.BROADTENXCellAnnotation>
Dimensions:             (cell: 5597650, L1_cat: 71, l1_tsne: 2, l1_umap: 2,
                         l2_tsne: 2, l2_umap: 2, l3_tsne: 2, l3_umap: 2,
                         mr_tsne: 2, mr_umap: 2)
Coordinates:
  * cell                (cell) object 'pBICCNsMMrACAiF019d210630A1TGACTCCAGGT...
  * L1_cat              (L1_cat) object '0' '1' '10' '11' ... '7' '70' '8' '9'
  * l1_tsne             (l1_tsne) <U9 'l1_tsne_0' 'l1_tsne_1'
  * l1_umap             (l1_umap) <U9 'l1_umap_0' 'l1_umap_1'
  * l2_tsne             (l2_tsne) <U9 'l2_tsne_0' 'l2_tsne_1'
  * l2_umap             (l2_umap) <U9 'l2_umap_0' 'l2_umap_1'
  * l3_tsne             (l3_tsne) <U9 'l3_tsne_0' 'l3_tsne_1'
  * l3_umap             (l3_umap) <U9 'l3_umap_0' 'l3_umap_1'
  * mr_tsne             (mr_tsne) <U9 'mr_tsne_0' 'mr_tsne_1'
  * mr_umap             (mr_umap) <U9 'mr_umap_0' 'mr_umap_1'
Data variables: (12/18)
    L1                  (cell) object '7' '7' '7' '7' ... '66' '66' '66' '66'
    L1_cat_annot        (L1_cat) object dask.array<chunksize=(71,), meta=np.ndarray>
    L2                  (cell) object '7_1' '7_1' '7_1' ... '66_5' '66_5' '66_5'
    L3                  (cell) object '7_1_9' '7_1_4' ... '66_5_4' '66_5_2'
    MajorRegionCluster  (cell) object dask.array<chunksize=(87464,), meta=np.ndarray>
    l1_tsne_coord       (cell, l1_tsne) float32 dask.array<chunksize=(174927, 1), meta=np.ndarray>
    ...                  ...
    mr_umap_coord       (cell, mr_umap) float32 dask.array<chunksize=(174927, 1), meta=np.ndarray>
    L1_annot            (cell) object 'ASC' 'ASC' 'ASC' ... 'ECPC' 'ECPC' 'ECPC'
    sample              (cell) object 'pBICCNsMMrACAiF019d210630A1' ... 'pBIC...
    DissectionRegion    (cell) object 'ACA' 'ACA' 'ACA' ... 'VIS' 'VIS' 'VIS'
    MajorRegion         (cell) object 'Isocortex' 'Isocortex' ... 'Isocortex'
    SubRegion           (cell) object 'ACA' 'ACA' 'ACA' ... 'VIS' 'VIS' 'VIS'
Attributes:
    L1_coord:                   ['l1_tsne', 'l1_umap']
    L2_coord:                   ['l2_tsne', 'l2_umap']
    L3_coord:                   ['l3_tsne', 'l3_umap']
    MajorRregionCluster_coord:  ['mr_tsne', 'mr_umap']
    annot_path:                 /mnt/home/BICCN/wmb/broad/BROAD.TENX.Annotati...
    cluster_data_var:           ['L1', 'L2', 'L3', 'MajorRegionCluster']
    cluster_hierarchy:          [['L1', 'L2', 'L3'], ['MajorRegionCluster']]

## L1 - Neuron Non-Neuron

In [5]:
rna_annot['L1_annot'].to_pandas().unique()

array(['ASC', 'BS', 'Cortex', 'ODC', 'ECPC', 'NonN', 'OPC', 'VLMC', 'CB',
       'OLF2', 'TH', 'THs', 'OLF1'], dtype=object)

In [6]:
# neuron (not include DG-GC and CB)
mc_neurons = ~mc_annot['L1_annot'].isin(['ODC', 'OPC', 'ASC', 'MGC', 'CB', 'CBX', 'DG'])
rna_neurons = ~rna_annot['L1_annot'].isin(['ODC', 'ASC', 'ECPC', 'NonN', 'OPC', 'VLMC', 'CB', 'OLF1'])

In [7]:
# pathlib.Path('../L1/Neuron').mkdir(exist_ok=True)
save_index(mc_annot.sel(cell=mc_neurons).get_index('cell'),
           file_name='../L1/Neuron/mc_cells.txt',
           downsample=10000000,
           random_state=0)

save_index(rna_annot.sel(cell=rna_neurons).get_index('cell'),
           file_name='../L1/Neuron/rna_cells.txt',
           downsample=10000000,
           random_state=0)

In [8]:
cwd = pathlib.Path().absolute()

with open('Snakefile') as f, open('../L1/Snakefile', 'w') as out_f:
    snakefile_tmp = f.read()
    snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'BROAD_TENX')
    snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L1'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
                                          str('L2'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
                                          str('L1_annot'))
    snakefile_tmp = snakefile_tmp.replace(
        'REPLACE_CATEGORICAL_KEY', str(['L1_annot', 'L2', 'DissectionRegion']))
    out_f.write(snakefile_tmp)

## L2

In [16]:
import joblib
import pathlib
import subprocess

In [17]:
mc_groups = pd.read_csv('../L1/Summary/mc_l1_inte_group.csv.gz',
                        index_col=0).squeeze()
rna_groups = pd.read_csv('../L1/Summary/rna_l1_inte_group.csv.gz',
                         index_col=0).squeeze()

In [18]:
for integration_group in rna_groups.unique():
    mc_cells = mc_groups[mc_groups == integration_group].index
    rna_cells = rna_groups[rna_groups == integration_group].index

    integration_group = f'{integration_group}'
    out_dir = pathlib.Path(f'../L2/{integration_group}')
    out_dir.mkdir(exist_ok=True, parents=True)

    save_index(mc_cells,
               f'../L2/{integration_group}/mc_cells.txt',
               downsample=10000000,
               random_state=0)
    save_index(rna_cells,
               f'../L2/{integration_group}/rna_cells.txt',
               downsample=10000000,
               random_state=0)

cwd = pathlib.Path().absolute()
with open('Snakefile') as f, open('../L2/Snakefile', 'w') as out_f:
    snakefile_tmp = f.read()
    snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'BROAD_TENX')
    snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L2'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
                                          str('L3'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
                                          str('L2'))
    snakefile_tmp = snakefile_tmp.replace(
        'REPLACE_CATEGORICAL_KEY', str(['L2', 'L3', 'DissectionRegion']))
    out_f.write(snakefile_tmp)

# L3

In [5]:
import joblib
import pathlib
import subprocess

In [6]:
L2_group = []
for path in pathlib.Path('../L2').glob('InteGroup*'):
    L2_group.append(str(path).split('/')[-1])

In [8]:
for l2_group in L2_group:
    mc_groups = pd.read_csv(f'../L2/{l2_group}/mc_integration_group.csv.gz',
                        index_col=0).squeeze()
    rna_groups = pd.read_csv(f'../L2/{l2_group}/rna_integration_group.csv.gz',
                         index_col=0).squeeze()
    
    for integration_group in rna_groups.unique():
        mc_cells = mc_groups[mc_groups == integration_group].index
        rna_cells = rna_groups[rna_groups == integration_group].index

        integration_group = f'{l2_group}_{integration_group}'
        if mc_cells.size <= 150 or rna_cells.size <= 150:
            print(integration_group)
        else:
            out_dir = pathlib.Path(f'../L3/{integration_group}')
            out_dir.mkdir(exist_ok=True, parents=True)

            save_index(mc_cells,
                       f'../L3/{integration_group}/mc_cells.txt',
                       downsample=1000000,
                       random_state=0)
            save_index(rna_cells,
                       f'../L3/{integration_group}/rna_cells.txt',
                       downsample=1000000,
                       random_state=0)


InteGroup2_12
InteGroup10_5
InteGroup10_8
InteGroup13_5
InteGroup13_4
InteGroup13_6
InteGroup22_1
InteGroup22_0
InteGroup22_2
InteGroup22_3
InteGroup9_9


In [5]:
cwd = pathlib.Path().absolute()
with open('Snakefile') as f, open('../L3/Snakefile', 'w') as out_f:
    snakefile_tmp = f.read()
    snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'BROAD_TENX')
    snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L3'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
                                          str('L3'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
                                          str('L3'))
    snakefile_tmp = snakefile_tmp.replace(
        'REPLACE_CATEGORICAL_KEY', str(['L3', 'DissectionRegion']))
    out_f.write(snakefile_tmp)

# L4

In [6]:
L2_group = []
for path in pathlib.Path('../L2').glob('InteGroup*'):
    L2_group.append(str(path).split('/')[-1])

In [7]:
for l2_group in L2_group:
    mc_groups = pd.read_csv(f'../L2/{l2_group}/mc_integration_group.csv.gz',
                        index_col=0).squeeze()
    rna_groups = pd.read_csv(f'../L2/{l2_group}/rna_integration_group.csv.gz',
                         index_col=0).squeeze()
    
    for integration_group in rna_groups.unique():
        mc_cells = mc_groups[mc_groups == integration_group].index
        rna_cells = rna_groups[rna_groups == integration_group].index

        integration_group = f'{l2_group}_{integration_group}'
        if mc_cells.size <= 150 or rna_cells.size <= 150:
            print(integration_group)
        else:
            out_dir = pathlib.Path(f'../L4/{integration_group}')
            out_dir.mkdir(exist_ok=True, parents=True)

            save_index(mc_cells,
                       f'../L4/{integration_group}/mc_cells.txt',
                       downsample=1000000,
                       random_state=0)
            save_index(rna_cells,
                       f'../L4/{integration_group}/rna_cells.txt',
                       downsample=1000000,
                       random_state=0)


InteGroup2_12
InteGroup10_5
InteGroup10_8
InteGroup13_5
InteGroup13_4
InteGroup13_6
InteGroup22_1
InteGroup22_0
InteGroup22_2
InteGroup22_3
InteGroup9_9


In [8]:
cwd = pathlib.Path().absolute()
with open('Snakefile') as f, open('../L4/Snakefile', 'w') as out_f:
    snakefile_tmp = f.read()
    snakefile_tmp = snakefile_tmp.replace('REPLACE_TEMPLATE_DIR', str(cwd))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_DATASET', 'BROAD_TENX')
    snakefile_tmp = snakefile_tmp.replace('REPLACE_CLUSTER_COL', str('L3'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_GROUP_KEY',
                                          str('L4'))
    snakefile_tmp = snakefile_tmp.replace('REPLACE_INTEGRATION_PLOT_KEY',
                                          str('L4'))
    snakefile_tmp = snakefile_tmp.replace(
        'REPLACE_CATEGORICAL_KEY', str(['L3', 'L4','DissectionRegion']))
    out_f.write(snakefile_tmp)